# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [285]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.gaussian_process import GaussianProcessClassifier

[nltk_data] Downloading package punkt to /Users/nbk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [286]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)
df.dropna(inplace=True)

X = df['message']
Y = df.loc[:, 'related':]

df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [287]:
def tokenize(text):
  text_ = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

  words = word_tokenize(text_)

  words = [w for w in words if w not in stopwords.words("english")]
  lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
  words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
  
  return words

In [288]:
print(X[2])
print(tokenize(X[2]))

Looking for someone but no name
['look', 'someone', 'name']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [289]:
clf = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
  ('vect', CountVectorizer(tokenizer=tokenize)),
  ('tfidf', TfidfTransformer()),
  ('clf', clf),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [290]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

X_train.shape

(7667,)

In [291]:
pipeline.fit(X_train, y_train)

/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x3373a7be0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [292]:
predicted = pipeline.predict(X_test)

In [293]:
def display_results(y_test, y_pred):
    results = pd.DataFrame(columns=['category', 'f1_score', 'precision', 'recall'])

    for idx, category in enumerate(y_test.columns):
        precision, recall, f1_score, _ = precision_recall_fscore_support(y_test[category], y_pred[:, idx], average='weighted')
        results.loc[idx] = [category, f1_score, precision, recall]
    
    print('Aggregated f1_score:', results['f1_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())

    return results

In [294]:
res = display_results(y_test, predicted)
res

/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

Aggregated f1_score: 0.9033298935850649
Aggregated precision: 0.895077486304793
Aggregated recall: 0.9248934967831681


/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

,category,f1_score,precision,recall
0,related,0.566178,0.567651,0.623239
1,request,0.567775,0.587681,0.641628
2,offer,0.998826,0.998436,0.999218
3,aid_related,0.540552,0.562396,0.599765
4,medical_help,0.908433,0.881547,0.937011
5,medical_products,0.947491,0.931533,0.964006
6,search_and_rescue,0.965510,0.954367,0.976917
7,security,0.976588,0.968946,0.984351
8,military,0.991206,0.988297,0.994131
9,child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [295]:
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__max_depth': [10, 30],
        'clf__estimator__min_samples_leaf': [2, 5, 10]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [296]:
cv.fit(X_train, y_train)
predicted_cv = cv.predict(X_test)

/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 't

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [297]:
res_cv = display_results(y_test, predicted_cv)
res_cv

/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

Aggregated f1_score: 0.8969832315225781
Aggregated precision: 0.8729856364369101
Aggregated recall: 0.9279038428099461


/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

,category,f1_score,precision,recall
0,related,0.509944,0.420265,0.648279
1,request,0.513409,0.423823,0.651017
2,offer,0.998826,0.998436,0.999218
3,aid_related,0.459758,0.369641,0.607981
4,medical_help,0.909411,0.881659,0.938967
5,medical_products,0.948078,0.931572,0.965180
6,search_and_rescue,0.965510,0.954367,0.976917
7,security,0.976588,0.968946,0.984351
8,military,0.991206,0.988297,0.994131
9,child_alone,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [298]:
clf_2 = MultiOutputClassifier(KNeighborsClassifier(3))

pipeline_2 = Pipeline([
  ('vect', CountVectorizer(tokenizer=tokenize)),
  ('tfidf', TfidfTransformer()),
  ('clf', clf_2),
])

pipeline_2.fit(X_train, y_train)

predicted_2 = pipeline_2.predict(X_test)
res_2 = display_results(y_test, predicted_2)
res_2


/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWa

Aggregated f1_score: 0.8999690179852811
Aggregated precision: 0.8906457805744912
Aggregated recall: 0.9163623717614328


/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nbk/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

,category,f1_score,precision,recall
0,related,0.519623,0.554178,0.509781
1,request,0.558135,0.550959,0.606416
2,offer,0.998826,0.998436,0.999218
3,aid_related,0.517051,0.515658,0.560642
4,medical_help,0.906471,0.881321,0.933099
5,medical_products,0.948022,0.937675,0.963615
6,search_and_rescue,0.964531,0.954323,0.974961
7,security,0.976588,0.968946,0.984351
8,military,0.991206,0.988297,0.994131
9,child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [299]:
pickle.dump(pipeline_2, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.